# Preprocessing

In this lab, we will be exploring how to preprocess tweets for sentiment analysis. We will provide a function for preprocessing tweets during this week's assignment, but it is still good to know what is going on under the hood. By the end of this lecture, you will see how to use the [NLTK](http://www.nltk.org) package to perform a preprocessing pipeline for Twitter datasets.

## install depend lib

In [8]:
!pip install nltk 

In [9]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random                              # pseudo-random number generator

## About the Twitter dataset

The sample dataset from NLTK is separated into positive and negative tweets. It contains 5000 positive tweets and 5000 negative tweets exactly. The exact match between these classes is not a coincidence. The intention is to have a balanced dataset. That does not reflect the real distributions of positive and negative classes in live Twitter streams. It is just because balanced datasets simplify the design of most computational methods that are required for sentiment analysis. However, it is better to be aware that this balance of classes is artificial. 

The dataset is already downloaded in the Coursera workspace. In a local computer however, you can download the data by doing:

In [10]:
# downloads sample twitter dataset. uncomment the line below if running on a local machine.
nltk.download('twitter_samples')

[nltk_data] Error loading twitter_samples: <urlopen error [WinError
[nltk_data]     10054] 远程主机强迫关闭了一个现有的连接。>


False

In [6]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

LookupError: 
**********************************************************************
  Resource [93mtwitter_samples[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('twitter_samples')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/twitter_samples[0m

  Searched in:
    - 'C:\\Users\\15962/nltk_data'
    - 'c:\\programdata\\anaconda3\\envs\\torch\\nltk_data'
    - 'c:\\programdata\\anaconda3\\envs\\torch\\share\\nltk_data'
    - 'c:\\programdata\\anaconda3\\envs\\torch\\lib\\nltk_data'
    - 'C:\\Users\\15962\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
